<a href="https://colab.research.google.com/github/marcelohfms/n8n/blob/main/Graficos_Relatorio_Performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: Now transform "worksheet" to dataFrame

import pandas as pd
from datetime import datetime, timedelta
import os

# Convert the list of lists to a Pandas DataFrame
df = pd.read_excel('relatorio_n8n.xlsx', sheet_name = 'Relatorio_em_Etapas')
operacao = pd.read_excel('relatorio_n8n.xlsx', sheet_name = 'Dados Resumo')
operacao = operacao[operacao['Informação'] == 'Operação']['Valor'].iloc[0]

# Now you have the worksheet data in a Pandas DataFrame called 'df'
# You can work with it like any other DataFrame
df.head()

# Get today's date in UTC-3
today_utc_minus_3 = datetime.utcnow() - timedelta(hours=3)
today = today_utc_minus_3.date()

In [ ]:
# prompt: Create a rename mapping for the columns in [Gap MKT', 'Gap PV', 'Gap Vendas', 'Gap MKT_up', 'Gap PV_up', 'Gap Vendas_up']

rename_mapping = {
    'Gap MKT': 'Gap MKT',
    'Gap PV': 'Gap Pre-Vendas',
    'Gap Vendas': 'Gap Vendas',
    'Gap MKT_up': 'MKT acima',
    'Gap PV_up': 'Pre-Vendas acima',
    'Gap Vendas_up': 'Vendas acima'
}

df = df.rename(columns=rename_mapping)

In [ ]:
gaps = [df['Potencial de Receita'].sum()]
labels = ['Potencial de Receita']
for col in ['Gap MKT', 'Gap Pre-Vendas', 'Gap Vendas', 'MKT acima', 'Pre-Vendas acima', 'Vendas acima']:
  gaps.append(df[col].sum())
  labels.append(col)
gaps.append(df['Potencial de Receita Realizado'].sum())
labels.append('Receita Realizada')

In [ ]:
# prompt: generate a waterfall graph from gaps and labels

import matplotlib.pyplot as plt

# Assuming gaps and labels are defined as in the previous code

# Calculate the cumulative sum of the gaps
cumulative_gaps = [sum(gaps[:i+1]) for i in range(len(gaps))]

def format_label(value):
  if abs(value) > 1_000_000:
    return f'{value / 1_000_000:.3f}M'
  elif abs(value) > 1_000:
    return f'{value / 1_000:.1f}k'
  else:
    return value

def waterfall_chart(iteration):
    # Create the waterfall chart
    fig, ax = plt.subplots(figsize=(14, 8))

    for i, (label, gap) in enumerate(zip(labels, gaps)):
        # Default color rules
        if i in [0, len(gaps) - 1]:
            color = 'grey'
        elif i <= 3:
            if i == iteration:
                color = 'red'
            elif iteration <= 3:
                color = 'lightpink'
            else:
                color = 'lightcoral'
        else:
            if i == iteration:
                color = 'midnightblue'
            else:
                color = 'skyblue'

        # Bottom position for bars
        bottom = 0 if i == len(gaps) - 1 else cumulative_gaps[i] - gap

        # Plot the bar
        ax.bar(label, gap, bottom=bottom, color=color)

        # Text styling for annotation
        text_color = 'white' if i == iteration and i > 3 else 'black'
        ax.text(label, bottom + gap / 2, format_label(gap), ha='center', va='center', color=text_color)

    # Remove the upper and right spines
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # Remove the y-axis
    ax.spines['left'].set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Chart title
    titles = ['', 'com Gap em Marketing', 'com Gap em Pre-Vendas', 'com Gap em Vendas',
              'acima da meta em MKT', 'acima da meta em Pre-Vendas', 'acima da meta em Vendas']
    ax.set_title('Funis de Marketing - Linhas ' + titles[iteration])
    folder = f"graficos_relatorio"
    os.makedirs(folder, exist_ok = True)
    fig.savefig(folder + f"/waterfall_chart_{iteration}_{today}_{operacao}.png", dpi=500, bbox_inches="tight")

    # Show the chart
    plt.show()


In [ ]:
for i in range(1,7):
  waterfall_chart(i)

## Gráficos Paretos

In [ ]:
gap_mkt = df[['Linha MKT', 'Gap MKT', 'Pareto_MKT']]
gap_pv = df[['Linha PV', 'Gap Pre-Vendas', 'Pareto_PV']]
gap_vendas = df[['Linha Vendas', 'Gap Vendas', 'Pareto_Vendas']]
acima_mkt = df[['Linha MKT_up', 'MKT acima', 'Pareto_MKT_up']]
acima_pv = df[['Linha PV_up', 'Pre-Vendas acima', 'Pareto_PV_up']]
acima_vendas = df[['Linha Vendas_up', 'Vendas acima', 'Pareto_Vendas_up']]

In [ ]:
# prompt: make a list with each item from the cell above
graph_list = [gap_mkt, gap_pv, gap_vendas, acima_mkt, acima_pv, acima_vendas]

In [5]:
def pareto_magic(df):
    pareto_col = [col for col in df.columns if 'Pareto' in col][0]
    linha_col = [col for col in df.columns if 'Linha' in col][0]
    gap_col = [col for col in df.columns if ('Pareto' not in col and 'Linha' not in col)][0]
    remaining_gap = df[df[pareto_col] > 0.8][1:][gap_col].sum()

    # Identificar até onde o corte será feito
    for row, _ in df.iterrows():
        if df.loc[row, pareto_col] > 0.8:
            df_new = df.iloc[:row+1].copy()
            break

    # Garantir consistência do DataFrame
    if remaining_gap != 0:
        last_row_data = {linha_col: 'Outras linhas', gap_col: remaining_gap, pareto_col: 1}
        df_new = pd.concat([df_new, pd.DataFrame([last_row_data])], ignore_index=True)

    # Garantir que os tipos de colunas sejam consistentes
    df_new[gap_col] = df_new[gap_col].astype(float)
    df_new[pareto_col] = df_new[pareto_col].astype(float)

    return df_new

In [ ]:
graph_list_processed = [pareto_magic(df) for df in graph_list]

In [ ]:
import matplotlib.pyplot as plt

# Assuming graph_list_processed[0] is defined as in the previous code

def pareto_graph(index, df):
  # Extract data for the plot
  pareto_col = [col for col in df.columns if 'Pareto' in col][0]
  linha_col = [col for col in df.columns if 'Linha' in col][0]
  gap_col = [col for col in df.columns if ('Pareto' not in col and 'Linha' not in col)][0]
  lines = df[linha_col]
  gap_mkt_values = abs(df[gap_col])
  pareto_mkt_values = df[pareto_col]

  # Create the combined line and bar plot
  fig, ax1 = plt.subplots(figsize=(14, 8))

  # Bar plot for Gap MKT
  bars = ax1.bar(lines, gap_mkt_values, color='midnightblue')

  # Set y-axis limits
  ax1.set_ylim([0, 2 * max(gap_mkt_values)])

  # Add data labels inside the bars
  for bar, value in zip(bars, gap_mkt_values):
      ax1.text(bar.get_x() + bar.get_width() / 2, bar.get_height() / 2,
              format_label(value), ha='center', va='center', color='white', fontsize=12, fontweight = 'bold')

  # Remove upper and right spines
  ax1.spines['top'].set_visible(False)
  ax1.spines['right'].set_visible(False)
  ax1.spines['left'].set_visible(False)

  # Create a second y-axis for the line plot
  ax2 = ax1.twinx()

  # Line plot for Pareto_MKT
  line = ax2.plot(lines, pareto_mkt_values, color='lightcoral', marker='o')

  # Add data labels as percentages to the top of the line
  for x, y in zip(lines, pareto_mkt_values):
      ax2.text(x, y + 0.05, f'{y*100:.0f}%', ha='center', va='bottom', color='black', fontsize=10,
             bbox=dict(facecolor='lightgrey', edgecolor='none', boxstyle='round', alpha=0.6))

  # Set y-axis limits for the line
  ax2.set_ylim([-0.4, 1.2])

  # Remove y-axis labels
  ax1.get_yaxis().set_visible(False)
  ax2.get_yaxis().set_visible(False)

  ax2.spines['top'].set_visible(False)
  ax2.spines['right'].set_visible(False)
  ax2.spines['left'].set_visible(False)

  # Add title
  titulos = ['no Gap de Marketing', 'no Gap de Pre-Vendas', 'no Gap de Vendas',
            'acima da meta em MKT', 'acima da meta em Pre-Vendas', 'acima da meta em Vendas']
  plt.title('Representatividade de cada linha ' + titulos[i])

  # Show the plot
  folder = f"graficos_relatorio"
  os.makedirs(folder, exist_ok = True)
  fig.savefig(folder + f"/pareto_chart_{index+1}_{today}_{operacao}.png", dpi=500, bbox_inches="tight")
  plt.show()


In [ ]:
for i, df in enumerate(graph_list_processed):
  pareto_graph(i, df)

In [ ]:
gaps_resumo = [gaps[0], gaps[1]+gaps[4], gaps[2]+gaps[5], gaps[3]+gaps[6], gaps[7]]
labels = ['Potencial de Receita', 'Efeito MKT', 'Efeito Pre-Vendas', 'Efeito Vendas', 'Receita Realizada']

In [ ]:
def final_waterfall_chart(gaps):
    # Create the waterfall chart
    fig, ax = plt.subplots(figsize=(14, 8))

    for i, (label, gap) in enumerate(zip(labels, gaps)):
        # Default color rules
        if i in [0, len(gaps) - 1]:
            color = 'grey'
        elif gaps[i] > 0:
            color = 'skyblue'
        else:
            color = 'lightcoral'

        cumulative_gaps = sum(gaps[:i+1])

        # Bottom position for bars
        bottom = 0 if i == len(gaps) - 1 else cumulative_gaps - gap

        # Plot the bar
        ax.bar(label, gap, bottom=bottom, color=color)

        # Text styling for annotation
        ax.text(label, bottom + gap / 2, format_label(gap), ha='center', va='center', color='black')

    # Remove the upper and right spines
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # Remove the y-axis
    ax.spines['left'].set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Chart title
    ax.set_title('Potencial de Receita - Funis de Marketing')
    folder = f"graficos_relatorio"
    os.makedirs(folder, exist_ok = True)
    fig.savefig(folder + f"/final_waterfall_chart_{today}_{operacao}.png", dpi=500, bbox_inches="tight")

    # Show the chart
    plt.show()

In [ ]:
final_waterfall_chart(gaps_resumo)